Supplement to "Dynamical moduli spaces and polynomial endomorphisms of configurations" by
Talia Blum, John R. Doyle, Trevor Hyde, Colby Kelln, Henry Talbott, and Max Weinreich.

This Sage notebook contains
1) A function to compute a Grobner basis for the ideal of the portrait moduli space \hat{M}_{P,Q,d}.
2) An example in degree 2 illustrating how to use this Grobner basis to calculate the dimension of the moduli space and, when the dimension is 0, the degree of the corresponding 0-dimensional affine scheme.
3) Lists of representatives of all quadratic and cubic portrait pairs on 4 and 6 points respectively.
4) Calculations of the number of such portraits obstructed by the interpolation and collision obstruction.
5) Details of the calculation described in the proof of Proposition 4.1.

In [2]:
#Given portraits P and Q on sets of size n represented by tuples,
#Returns an ideal in variables xk with 0 <= k < n describing the relations on these variables
#imposed by the existence of degree at most d polynomials f(x) and g(x) realizing f and g.
#Note that if P = (0,0,1,2), this represents the function P: {0,1,2,3} --> {0,1,2,3} 
#defined by P(0) = P(1) = 0, P(2) = 1, P(3) = 2.
def portrait_ideal(d, P, Q):
    
    #Number of points in the portrait is same as length of f as a tuple.
    n = len(f)
    
    #Create variables for the points and for the coefficients of the polynomial f.
    point_vars = [v('x',k) for k in range(n)]
    coeff_vars = [v('f',k) for k in range(d + 1)] + [v('g',k) for k in range(d + 1)]
    
    #Construct a polynomial ring with these variables.
    #The variable z is used to invert product of xi - xj
    R = PolynomialRing(QQ, point_vars + coeff_vars + [var('z')])
    #This is a technical step required to make the ring R recognize its variables by their names.
    R.inject_variables(verbose = False)
    
    #Construct the interpolation relations.
    relations = [sum([v('f',j)*v('x',k)^j for j in range(d + 1)]) - v('x',P[k]) for k in range(n)]
    relations += [sum([v('g',j)*v('x',k)^j for j in range(d + 1)]) - v('x',Q[k]) for k in range(n)]
    
    #This relation inverts the product of xi - xj.
    relations += [z*prod(v('x',i) - v('x',j) for i in range(n) for j in range(n) if i < j) - 1]
    
    #This relation eliminates the affine symmetry.
    relations += [x0, x1 - 1]
    
    #Eliminate the coefficient variables and z to get ideal of relations on the configurations.
    config_relations = (R*(relations)).elimination_ideal([R(a) for a in coeff_vars] + [R(var('z'))])
    
    #Construct polynomial ring in just the xi variables.
    S = PolynomialRing(QQ,point_vars)
    S.inject_variables(verbose = False)
    
    #Return ideal of S generated by the relations on the xi variables imposed by the portraits having
    #degree at most d realizations.
    return S*config_relations.gens()

#Check if portrait is admissible in degree d
def is_admissible(d, f):
    
    n = len(f)
    
    #Check that f is at most d to 1
    for k in range(n):
        if sum([f[i] == k for i in range(n)]) > d:
            return False
        
    #Compute a tallied list of length of cycles
    function_graph = DiGraph([(i,f[i]) for i in range(n)], loops = True)
    cycle_length_tally = tally([len(cycle) - 1 for cycle in function_graph.all_simple_cycles()])
    for cyc_count in cycle_length_tally:
        #j is the length of the cycle
        j = cyc_count[0]
        #If f has more j cycles than possible for a degree d polynomial, return false.
        if cyc_count[1] > sum([moebius(i)*d^(j/i) for i in divisors(j)])/j:
            return False
    #Otherwise return True
    return True

#HELPER FUNCTIONS

#helper function for making variables with a given index
def v(name,index):
    return var(name + '%d' % index)

#Converts a partition given as a list of parts into a list of (j, m_j)
#where m_j is the number of parts of size j in the partition.
def tally(partition):
    values = list(set(partition))
    return [(a,partition.count(a)) for a in values]

Example: Calculation of a portrait moduli space for quadratic portraits on 4 points.

In [26]:
#Let f and g be the following two portraits on 4 points.
P = (0,0,1,2)
Q = (2,1,1,3)

In [27]:
#First we check that both portraits are admissible in degree 2.
print is_admissible(2,P)
print is_admissible(2,Q)

True
True


In [31]:
#Next we compute a Grobner basis for the ideal of relations imposed on x0, x1, x2, x3
#by an affine equivalence class of configuration of 4 points simultaneously supporting 
#degree at most 2 realizations of f and g.
M_PQ2 = portrait_ideal(2,P,Q)
print 'Ideal generators: ' + str(M_PQ2.gens())

Ideal generators: [x1 - 1, x0, x3^2 - 2*x2 - 4*x3 + 4, x2*x3 - 2*x2 - 2*x3 + 2, x2^2 - 2*x2 - x3 + 2]


In [36]:
#Having a Grobner basis for this ideal makes calculations of dimension and degree fast.
print 'Dimension: ' + str(M_PQ2.dimension())
print 'Degree: ' + str(M_PQ2.vector_space_dimension())

Dimension: 0
Degree: 3


In [38]:
#We see that this portrait moduli space is 0-dimensional and contains 3 points.
#We can compute complex approximations of these 3 configurations explicitly as follows:
M_PQ2.variety(CC)

[{x2: 2.83928675521416, x1: 1.00000000000000, x0: 0.000000000000000, x3: 4.38297576790624},
 {x2: 0.580356622392919 + 0.606290729207199*I, x1: 1.00000000000000, x0: 0.000000000000000, x3: 0.808512116046881 - 0.508851778832738*I},
 {x2: 0.580356622392919 - 0.606290729207199*I, x1: 1.00000000000000, x0: 0.000000000000000, x3: 0.808512116046881 + 0.508851778832738*I}]

Example: Counting obstructed portrait pairs.

In [3]:
#A complete list of representatives of admissble portait pairs (P,Q) for degree 2 and 4 points,
#and degree 3 and 6 points may be created by generating a list of all pairs of admissible degree d portraits,
#and then removing pairs which are duplicate after conjugating by some permutation.
#For convenience, we have included these lists in the following files:
portraits_d2n4 = load('portraits_d2n4.sobj')
portraits_d3n6 = load('portraits_d3n6.sobj')

In [59]:
#We can check the number of portrait pairs obstructed by the interpolation obstruction as follows:

print 'The number of quadratic pairs obstructed by interpolation is'
print sum([sum([P[i] == Q[i] for i in range(4)]) > 2 for (P,Q) in portraits_d2n4])
print
print 'The number of cubic pairs obstructed by interpolation is'
print sum([sum([P[i] == Q[i] for i in range(6)]) > 3 for (P,Q) in portraits_d3n6])

The number of quadratic pairs obstructed by interpolation is
39

The number of cubic pairs obstructed by interpolation is
12733


In [77]:
#We can check the number of portrait pairs obstructed by the collision obstruction as follows:

import itertools as it
print 'The number of quadratic pairs obstructed by collision is'
print sum([1 == sum([P[i] == P[j] and Q[i] == Q[j] for (i,j) in it.combinations(range(4),2)]) for (P,Q) in portraits_d2n4])
print
print 'The number of cubic pairs obstructed by collision is'
collision_obstruction_d3n6 = 0
for (P,Q) in portraits_d3n6:
    common = sum([P[i] == P[j] and Q[i] == Q[j] for (i,j) in it.combinations(range(6),2)])
    if common >= 2 and common < 6:
        collision_obstruction_d3n6 += 1
print collision_obstruction_d3n6


The number of quadratic pairs obstructed by collision is
133

The number of cubic pairs obstructed by collision is
39519


Calculations for Proposition 4.1

In [14]:
#First we calculate the dimension of the portrait moduli spaces M_{P,Q,2} for all quadratic pairs (P,Q)
quadratic_dimensions = []
for (P,Q) in portraits_d2n4:
    ideal = portrait_ideal(2,P,Q)
    dimension = ideal.dimension()
    quadratic_dimensions.append(((P,Q),dimension,ideal))

In [15]:
#Print the list of pairs with a one dimensional moduli space
for (a,b,c) in quadratic_dimensions:
    if b == 1:
        print a

((0, 0, 1, 1), (0, 0, 2, 2))
((0, 0, 1, 1), (1, 1, 0, 0))
((0, 0, 1, 1), (1, 1, 2, 2))
((0, 0, 1, 1), (2, 2, 0, 0))
((0, 0, 1, 1), (2, 2, 1, 1))
((0, 0, 1, 1), (2, 2, 3, 3))
((0, 0, 1, 3), (0, 2, 3, 3))
((0, 0, 2, 2), (0, 0, 3, 3))
((0, 0, 2, 2), (1, 1, 3, 3))
((0, 0, 2, 2), (2, 2, 0, 0))
((0, 0, 2, 2), (2, 2, 1, 1))
((0, 0, 2, 2), (3, 3, 1, 1))
((1, 0, 0, 1), (1, 3, 3, 1))
((1, 0, 0, 1), (2, 3, 3, 2))


In [ ]:
#Observe that for all but one pair (P,Q), both P and Q are two-image portraits.
#In the one exception ((0, 0, 1, 3), (0, 2, 3, 3)), neither portrait is two-image.
#Therefore, if there are at least 3 distinct portraits P1, P2, P3 for which
#(Conf_{P1,2} \cap Conf_{P2,2} \cap Conf_{P3,2})/Aff_1 is one dimensional,
#it follows that each Pi is a two-image portrait.

In [61]:
#Next we compute a list of representatives of affine equivalence classes of configurations of 4 points
#which have at least 2 quadratic endomorphisms that are not realizations of two-image portraits
dimension_0 = [c for (a,b,c) in quadratic_dimensions if b == 0]
configs = []
K = ComplexField(50)
for ideal in dimension_0:
    configs += ideal.variety(K)
configs = set(tuple(q.values()) for q in configs)

In [67]:
#This function interpolates all set-theoretic endomorphisms of a configuration and records their degree.
import itertools as it
def config_endomorphisms(q):
    n = len(q)
    endos = []
    funs = it.product(range(n),repeat = n)
    R = K['x']
    deg_dict = {d:0 for d in range(-1,n)}
    for fun in funs:
        f = R.lagrange_polynomial([(q[i],q[fun[i]]) for i in range(n)])
        non_zero_coeffs = [b.norm() > .0001 for b in f.coefficients(sparse = False)]
        deg1 = f.degree()
        if deg1 >= 0:
            deg = max([i for i in range(deg1+1) if non_zero_coeffs[i]] + [-1])
            deg_dict[deg] += 1
    return deg_dict

In [70]:
#List the configurations together with the number of quadratic endomorphisms.
quadratic_endos = []
for q in configs:
    quadratic_endos.append((config_endomorphisms(q)[2],q))
for a in reversed(sorted(quadratic_endos)):
    print a

(28, (1.0000000000000 + 1.0000000000000*I, 1.0000000000000, 0.00000000000000, 1.0000000000000*I))
(28, (1.0000000000000 - 1.0000000000000*I, 1.0000000000000, 0.00000000000000, -1.0000000000000*I))
(28, (0.50000000000000 + 0.50000000000000*I, 1.0000000000000, 0.00000000000000, 0.50000000000000 - 0.50000000000000*I))
(28, (0.50000000000000 - 0.50000000000000*I, 1.0000000000000, 0.00000000000000, 0.50000000000000 + 0.50000000000000*I))
(28, (1.0000000000000*I, 1.0000000000000, 0.00000000000000, 1.0000000000000 + 1.0000000000000*I))
(28, (-1.0000000000000*I, 1.0000000000000, 0.00000000000000, 1.0000000000000 - 1.0000000000000*I))
(20, (2.4142135623731, 1.0000000000000, 0.00000000000000, 3.4142135623731))
(20, (1.7071067811865, 1.0000000000000, 0.00000000000000, -0.70710678118655))
(20, (1.4142135623731, 1.0000000000000, 0.00000000000000, 0.41421356237309))
(20, (0.70710678118655, 1.0000000000000, 0.00000000000000, 0.29289321881345))
(20, (0.29289321881345, 1.0000000000000, 0.00000000000000

In [ ]:
#Each of the configurations with 28 quadratic endomorphisms is affine equivalent to the 4th roots of unity.